In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import math

**1. Estimate the integral by simulation (the crude Monte Carlo estimator). Use eg. an estimator based on 100 samples and present the result as the point estimator and a confidence interval.**


In [2]:
u = np.random.uniform(size=100)
y = np.exp(u)
print(f'The crude Monte Carlo estimator: {sum(y)/len(y)}')
print(f'Variance: {np.var(y)}')
print(f'Confidence Interval: {st.t.interval(0.95, len(y)-1, loc=np.mean(y), scale=st.sem(y))}')

The crude Monte Carlo estimator: 1.6855923521453624
Variance: 0.21286099221965205
Confidence Interval: (1.5935856213811088, 1.7775990829096173)


**2. Estimate the integral using antithetic variables, with comparable computer ressources.**

In [3]:
u = np.random.uniform(size=100)
y = (np.exp(u) + np.exp(1-u)) / 2
print(f'Estimator with Antithetic Variables: {sum(y)/len(y)}')
print(f'Variance: {np.var(y)}')
print(f'Confidence Interval: {st.t.interval(0.95, len(y)-1, loc=np.mean(y), scale=st.sem(y))}')

Estimator with Antithetic Variables: 1.7130601352043109
Variance: 0.0038336112935857107
Confidence Interval: (1.7007127329695086, 1.7254075374391131)


**3. Estimate the integral using a control variable, with comparable computer ressources.**


In [4]:
u = np.random.uniform(size=100)
x = np.exp(u)
y = u
cov = np.mean(x*y) - (np.mean(x)*np.mean(y))
c = -cov / np.var(y)
mu_y = np.mean(y)
z = x + c*(y-mu_y)
print(f'Estimator with Control Variable: {sum(z)/len(z)}')
print(f'Variance: {np.var(z)}')
print(f'Confidence Interval: {st.t.interval(0.95, len(z)-1, loc=np.mean(z), scale=st.sem(z))}')

Estimator with Control Variable: 1.6968809461356462
Variance: 0.003966635601629877
Confidence Interval: (1.684321146506549, 1.7094407457647411)


**4. Estimate the integral using stratified sampling, with comparable computer ressources.**


In [5]:
def rd():
    return np.random.uniform()

y = []
for i in range(100):
    num = np.exp(rd()/5) + np.exp(1/5+rd()/5) + np.exp(2/5+rd()/5) + np.exp(3/5+rd()/5) + np.exp(4/5+rd()/5)
    y.append(num / 5)
y = np.asarray(y)
print(f'Estimator with Stratified Sampling: {sum(y)/len(y)}')
print(f'Variance: {np.var(y)}')
print(f'Confidence Interval: {st.t.interval(0.95, len(y)-1, loc=np.mean(y), scale=st.sem(y))}')

Estimator with Stratified Sampling: 1.724305330012399
Variance: 0.0024568404092535235
Confidence Interval: (1.7144207087962686, 1.7341899512285293)


**5. Use control variates to reduce the variance of the estimator in exercise 4 (Poisson arrivals).**

In [6]:
class Blocking_System:
    def __init__(self, m: int, service, arrival, 
                 arrival_method: str = 'poisson', service_method: str = 'exp',
                 ):
        self.generate_cv()
        self.clock=0.0                      #simulation clock
        self.num_service_units=m            #system with m service units
        self.arrival_mtd=arrival_method     #arrival distribution
        self.service_mtd=service_method     #service distribution
        self.param_service=service          #parameter for service dist.
        self.param_arrival=arrival          #parameter for arrival dist.
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.gen_arrival_time()   #time of next arrival
        self.t_departures=np.ones(m)*100000. #departure times for each service unit (100.000 as infinite)
        self.dep_sums=np.zeros((m,), dtype=int) #Sum of service time
        self.states=np.zeros((m,), dtype=int) #current states
        self.num_of_departures=np.zeros((m,), dtype=int) #number of customers served
        self.lost_customers=0               #customers who left without service
        self.num_in_system=0                #customers in the system
    

    def generate_cv(self):
        # Control variates impl.
        u = np.random.uniform(size=100)
        x = np.random.poisson(size=100)
        y = u
        cov = np.mean(x*y) - (np.mean(x)*np.mean(y))
        c = -cov / np.var(y)
        mu_y = np.mean(y)
        self.poisson_c=c
        self.poisson_mu_y=mu_y


    def time_adv(self):                                                       
        t_departure=min(self.t_departures)
        idx = list(self.t_departures).index(t_departure)
        if self.t_arrival<t_departure:
            self.clock=self.t_arrival
            self.arrival()
        else:
            self.clock=t_departure
            self.departure(idx)


    def arrival(self):              
        self.num_arrivals += 1
        self.num_in_system += 1

        accepted = False
        for idx in range(self.num_service_units):
            if self.states[idx]==0:
                accepted = True
                dep=self.gen_service_time()
                self.dep_sums[idx] += dep
                self.t_departures[idx]=self.clock + dep
                self.states[idx]=1
                break

        self.t_arrival=self.clock+self.gen_arrival_time()
        if not accepted:
            self.lost_customers += 1


    def departure(self, idx: int):
        self.num_of_departures[idx] += 1
        self.t_departures[idx]=100000. # (100.000 as infinite)
        self.states[idx]=0                  


    def gen_arrival_time(self):         #function to generate arrival times 
        if self.arrival_mtd=='erlang':
            return (np.random.gamma(self.param_arrival)) # Erlang distribution (using Gamma with shape=int)
        elif self.arrival_mtd=='hyperexp': # Hyper Exponential distribution p1 = 0.8, λ1 = 0.8333, p2 = 0.2, λ2 = 5.0
            if np.random.uniform() <= 0.8: #p1
                return (np.random.exponential(scale=1./0.833)) #λ1
            else: #p2
                return (np.random.exponential(scale=1./5.)) #λ2
        elif self.arrival_mtd=='cv-poisson':
            y = np.random.uniform()
            x = np.random.poisson()
            return x + self.poisson_c*(y - self.poisson_mu_y)
        else:
            return (np.random.poisson()) # Poisson distribution

    
    def gen_service_time(self):         #function to generate service time
        if self.service_mtd=='constant':
            return self.param_service
        if self.service_mtd=='pareto': # Pareto distribution
            return (np.random.pareto(self.param_service))
        if self.service_mtd=='normal':
            return (np.random.normal(loc=self.param_service)) # Normal Distribution
        else:
            return (np.random.exponential()) # Exponential distribution (lamda=1)
    
    

In [7]:
results = pd.DataFrame([],columns=['run','mean','count','std','CI low limit','CI high limit','CI range'])

In [8]:
m = 10
mu_service = 8
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival)
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival)
    while s.num_in_system <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df  

,Fraction of blocked customers,Average interarrival time,Total Customers,Blocked Customers
0,0.0005,0.995300,10001.0,5.0
1,0.0001,0.990801,10001.0,1.0
2,0.0002,0.995000,10001.0,2.0
3,0.0000,1.005999,10001.0,0.0
4,0.0000,0.987201,10001.0,0.0
5,0.0000,1.019998,10001.0,0.0
6,0.0002,1.002600,10001.0,2.0
7,0.0001,0.985501,10001.0,1.0
8,0.0004,0.989201,10001.0,4.0
9,0.0001,1.002000,10001.0,1.0


In [9]:
stats = df['Fraction of blocked customers'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []
m = stats.loc['mean']
c = stats.loc['count']
s = stats.loc['std']
ci95_hi.append(m + 1.95*s/math.sqrt(c))
ci95_lo.append(m - 1.95*s/math.sqrt(c))

stats['CI low limit'] = ci95_lo
stats['CI high limit'] = ci95_hi
stats['CI range'] = ci95_hi[0] - ci95_lo[0]

stats['run'] = 'Standard Poisson'
results = results.append(stats,ignore_index=True)

In [10]:
m = 10
mu_service = 8
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival, arrival_method='cv-poisson')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival, arrival_method='cv-poisson')
    while s.num_in_system <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df  

,Fraction of blocked customers,Average interarrival time,Total Customers,Blocked Customers
0,0.0000,0.985719,10001.0,0.0
1,0.0000,1.016801,10001.0,0.0
2,0.0000,0.996130,10001.0,0.0
3,0.0000,0.977758,10001.0,0.0
4,0.0003,1.000607,10001.0,3.0
5,0.0000,1.021193,10001.0,0.0
6,0.0002,1.006060,10001.0,2.0
7,0.0001,0.974552,10001.0,1.0
8,0.0000,0.989724,10001.0,0.0
9,0.0000,0.994879,10001.0,0.0


In [11]:
stats = df['Fraction of blocked customers'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []
m = stats.loc['mean']
c = stats.loc['count']
s = stats.loc['std']
ci95_hi.append(m + 1.95*s/math.sqrt(c))
ci95_lo.append(m - 1.95*s/math.sqrt(c))

stats['CI low limit'] = ci95_lo
stats['CI high limit'] = ci95_hi
stats['CI range'] = ci95_hi[0] - ci95_lo[0]

stats['run'] = 'Poisson with Control Variates'
results = results.append(stats,ignore_index=True)

In [12]:
results

,run,mean,count,std,CI low limit,CI high limit,CI range
0,Standard Poisson,0.00016,10.0,0.000171,[5.438206253347918e-05],[0.0002655859406662009],0.000211
1,Poisson with Control Variates,0.00006,10.0,0.000107,[-6.286625014204773e-06],[0.0001262746262140848],0.000133


As it can be checked in the table above, the CI range of values has been reduced from 0.000211 to 0.000133, 37% less.

**6. Demonstrate the effect of using common random numbers in exercise 4 for the difference between Poisson arrivals (Part 1) and a renewal process with hyperexponential interarrival times. Remark: You might need to some thinking and some re-programming.**

1.1449e-08

**7. Use importance sampling with g(x) = λ exp (−λ ∗ x) to calculate the integral. Try to find the optimal value of λ by calculating the variance of h(x)f(x)/g(x) and verify by simulation. Note that importance sampling with the exponential distribution will not reduce the variance in this case compared to the other methods.**

In [13]:
u = np.random.uniform(size=100)
x = np.exp(u)
lamda = 1 # at least for the combination of f and g this is not affecting
f_x = lamda*np.exp(-lamda*x)
g_x = lamda*np.exp(-lamda*x) + 0.1 # if we increase this difference the variance decreases
z = (x*f_x) / g_x
print(f'Estimator with Importance Sampling: {sum(z)/len(z)}')
print(f'Variance: {np.var(z)}')
print(f'Confidence Interval: {st.t.interval(0.95, len(z)-1, loc=np.mean(z), scale=st.sem(z))}')

Estimator with Importance Sampling: 1.0435444883162455
Variance: 0.011059429800013813
Confidence Interval: (1.02257258115986, 1.0645163954726313)


In [23]:
u = np.random.uniform(size=100)
x = np.exp(u)
lamda = 1

def h_x(x):
    if x > 1:
        return 0
    else:
        return 1


h = np.asarray([h_x(i) for i in x])
mean = np.mean(h)
print(x)
print(h)
g = lamda*np.exp(-lamda*x)
z = np.mean(h*x / g)
print(z)

print(f'Estimator with Importance Sampling: {z}')
print(f'Variance: {np.var(z)}')
print(f'Confidence Interval: {st.t.interval(0.95, len(z)-1, loc=np.mean(z), scale=st.sem(z))}')

[1.90642585 2.36946353 2.55009376 1.15121777 1.15902882 1.02685041
 1.8990347  1.28331772 1.74142357 1.70473704 1.39221983 1.84212246
 2.28526663 1.96938152 2.35686827 1.7741569  1.71464942 1.37465734
 1.07685105 1.11811725 1.00608393 1.2950797  1.3980464  2.21546044
 1.3519062  1.83922898 2.21926637 1.90542909 2.59409622 1.85398186
 2.4847739  2.37762259 2.49475227 1.26471865 1.92275753 1.51327183
 2.40565994 1.93916603 1.20356315 1.52137216 1.91470005 1.49811363
 1.43113728 1.77714579 1.00104115 1.24338406 1.32818509 2.61513455
 1.72338037 1.29177825 2.16342383 1.21137734 1.75281807 1.02150618
 1.77649234 2.48682038 1.89878778 1.63680307 1.93354865 1.00238455
 2.3456893  1.55128348 1.55000097 2.00186635 1.83081921 1.29777549
 1.33263927 1.97983054 2.24631015 1.96996561 2.52832973 1.50405603
 1.48277099 1.37966999 1.02280022 2.14943261 1.50631028 1.03015442
 2.47912658 2.44073465 1.20202484 2.64474198 1.18914706 1.12017666
 2.54371944 1.24794961 1.59953451 2.50640788 1.36315345 1.1248

TypeError: 'numpy.float64' object is not iterable

**8. Estimate the probability X > a. For a standard normal random variable Z ∼ N(0, 1) using the crude Monte Carlo estimator. Then try importance sampling with a normal density with mean a and
variance σ2. For the expirements start using σ2 = 1, use different values of a (e.g. 2 and 4), and different sample sizes. If time permits experiment with other values for σ2. Finally discuss the efficiency of the methods.**

'f_x = lamda*np.exp(−lamda*x)'

**9. For the Pareto case derive the IS estimator for the mean using the first moment distribution as sampling distribution. Is the approach meaningful? and could this be done in general?**